# This will be the demo integration of Colivara with LLamaIndex

#### First we need to install the dependancies

In [1]:
%pip install llama-index-llms-openai -q
%pip install llama-index-multi-modal-llms-openai -q
%pip install -U llama-index-core -q
%pip install -U colivara-py -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


#### Import the required libraries


In [2]:
# utilities
import base64 # for converting docs binaries to base64
from pathlib import Path 
import getpass
import os 
import requests 

from colivara_py import ColiVara 

# llamaindex tools
from llama_index.multi_modal_llms.openai import OpenAIMultiModal
from llama_index.core.schema import ImageNode


In [3]:
# Setup Environment Variables
os.environ['OPENAI_API_KEY'] = getpass.getpass()
os.environ['COLIVARA_API_KEY'] = getpass.getpass()


#### Download example docs for our demo (you can use your own docs as well)

In [4]:
def download_file(url, local_filename):
    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Ensure the 'docs' directory exists
        os.makedirs("docs", exist_ok=True)

        # Write the content to a local file
        with open(local_filename, "wb") as f:
            f.write(response.content)
        print(f"Successfully downloaded: {local_filename}")
    else:
        print(f"Failed to download: {url}")


# URLs and local filenames
files = [
    {
        "url": "https://github.com/tjmlabs/colivara-demo/raw/main/docs/Work-From-Home%20Guidance.pdf",
        "filename": "docs/Work-From-Home-Guidance.pdf",
    },
    {
        "url": "https://github.com/tjmlabs/colivara-demo/raw/main/docs/StaffVendorPolicy-Jan2019.pdf",
        "filename": "docs/StaffVendorPolicy-Jan2019.pdf",
    },
]

In [5]:
# Download each file
for file in files:
    download_file(file["url"], file["filename"])


Successfully downloaded: docs/Work-From-Home-Guidance.pdf
Successfully downloaded: docs/StaffVendorPolicy-Jan2019.pdf


#### Start Colivara Client and Create Collection `to upsert documents into`.

In [6]:
rag_client = ColiVara(
    base_url="https://api.colivara.com", api_key=os.environ["COLIVARA_API_KEY"]
)

In [7]:
rag_client.create_collection("llama-index-demo")

CollectionOut(id=77, name='llama-index-demo', metadata={}, num_documents=0)

#### Upsert the documents to the collection

In [8]:
def sync_documents():
    # get all the documents under docs/ folder and upsert them to colivara
    documents_dir = Path("docs")
    files = list(documents_dir.glob("**/*"))

    for file in files:
        with open(file, "rb") as f:
            file_content = f.read()
            encoded_content = base64.b64encode(file_content).decode("utf-8")
            rag_client.upsert_document(
                name=file.name,
                document_base64=encoded_content,
                collection_name="llama-index-demo",
                wait=True,
            )
            print(f"Upserted: {file.name}")

Internet connection has a great impact on this step.

In [9]:
sync_documents()

Upserted: Work-From-Home-Guidance.pdf
Upserted: StaffVendorPolicy-Jan2019.pdf


There is a step called query transformation/translation, which is the process of converting the user query into a format that the RAG model can understand. 
This step is necessary because the RAG model expects the input to be in a specific format.

***for the purpose of the demo, we will skip it and use the RAG query as the input to the RAG directly.***

Setup ended successfully.
Let's begin work.


In [10]:
query = "What is the work from home policy?"

you can do search by just using this code snippet:

```python
rag_client.search(query="some query",collection_name="collection_name", top_k=5)
``` 

In [ ]:
## you can try performing search to see results object structure

# results = rag_client.search(query, collection_name="llama-index-demo")
# print(results.results)

In [11]:
# get context using the RAG
def get_context(query):
    results = rag_client.search(query=query, collection_name="llama-index-demo")
    results = results.results
    context = []
    for result in results:
        base64 = result.img_base64
        context.append(ImageNode(image=base64))
    return context

In [12]:
context = get_context(query)

Now build the prompt using the context and query, and initialize the chat model.


In [13]:
openai_api_key = os.getenv("OPENAI_API_KEY")
gpt_4o = OpenAIMultiModal(model="gpt-4o", max_new_tokens=500, api_key=openai_api_key, temperature=0)

In [14]:
complete_response = gpt_4o.complete(query, context)

In [15]:
print(complete_response)

The work from home policy for Mount Sinai Health System includes the following key points:

1. **Remote Work Arrangements**: 
   - Remote work is considered a viable alternative when it benefits both the organization and the employee.
   - Employees must have a formal agreement to work remotely, specifying the terms and conditions.

2. **Remote Work Schedule**:
   - Employees must agree on a specific schedule, including days and work hours.
   - Additional requirements, such as checking emails or contacting managers, may be specified.

3. **Equipment**:
   - No equipment can be removed from Mount Sinai without authorization.

4. **Evaluation**:
   - Employees must participate in evaluation surveys and focus groups regarding remote work.

5. **Term and Termination**:
   - The agreement can be terminated by Mount Sinai at any time for any reason.

6. **Legal Compliance**:
   - The agreement is subject to New York law and applicable federal laws.

Employees and managers must sign the agre

## That's it for now. You can continue to explore more features and capabilities of LlamaIndex.